In [1]:
import h5py

from multiprocessing import Pool
import time

import numpy as np
import scipy

import warnings

np.random.seed(7)

masterdir = '/home/seismoai/data/forecasting/'
blockdir = masterdir + 'blockdata/'

# 0. Load data and initialize global functions

In [2]:
npzfile = np.load(masterdir + 'Japan_ISC.npz', allow_pickle=True)
catalog_mag = npzfile['mag']
catalog_dat = npzfile['dat']
catalog_lon = npzfile['lon']
catalog_lat = npzfile['lat']
catalog_dep = npzfile['dep']

In [3]:
def calcBall(bmags):  
    if len(bmags) < 2: 
        return np.nan
    b1 = np.sum(bmags)/len(bmags) - np.min(bmags)
    if b1 == 0.0: 
        return np.nan 
    return 1.0/b1*np.log10(np.e)

def calcBlim(bmags, bmin=3):  
    bmags = bmags[~np.isnan(bmags)]
    bmags = bmags[bmags>=bmin]
    if len(bmags) < 2: 
        return np.nan
    b1 = np.sum(bmags)/len(bmags) - np.min(bmags)
    if b1 == 0.0: 
        return np.nan 
    return 1.0/b1*np.log10(np.e)

def calcBplus(bmags):  
    if len(bmags) < 2: 
        return np.nan
    bmags = bmags[1:][np.diff(bmags) >= 0.0]
    if len(bmags) < 2: 
        return np.nan
    b1 = np.sum(bmags)/len(bmags) - np.min(bmags)
    if b1 == 0.0: 
        return np.nan 
    return 1.0/b1*np.log10(np.e)

In [4]:
def processDayBlock(e):
    # make selection
    te = time.time()
    sel = np.logical_and(
            np.logical_and(
                catalog_dat > e - np.timedelta64(lookback_gl, 'D'),
                catalog_dat < e - np.timedelta64(0, 'D')),
            np.logical_and(catalog_dep/1000 <= depth1, 
                           catalog_dep/1000 > depth0))
    
    sel = np.logical_and(sel, catalog_mag == catalog_mag) # Filter out nan Magnitudes
    s_long = catalog_lon[sel]
    s_lats = catalog_lat[sel]
    s_mags = catalog_mag[sel]
    s_date = catalog_dat[sel]
    s_depth = catalog_dep[sel]
    s_coord = np.array([s_long, s_lats]).T.astype(float)
    
    #calculate points:
    px, py = np.meshgrid(np.arange(long_min+gridsize/2, long_max, gridsize), np.arange( lat_min+gridsize/2,  lat_max, gridsize))
    points = np.reshape(np.append(px, py), (-1, 1, 2), order='F')
    
    res_n_lon = int((long_max - long_min) / gridsize)
    res_n_lat = int((lat_max - lat_min) / gridsize)
    
    b_values = np.zeros((res_n_lon*res_n_lat), dtype='float16')
    n_values = np.zeros((res_n_lon*res_n_lat), dtype='int')

    t0 = time.time()

    for k, p in enumerate(points): # 29s
        sel_points = np.linalg.norm(p-s_coord, axis=1) < dr_gl
        k_mags = s_mags[sel_points]
        b_values[k] = calcB(k_mags)
        n_values[k] = len(k_mags)
        
    t1 = time.time()
    
    b_values = np.reshape(b_values, (res_n_lon,res_n_lat))
    n_values = np.reshape(n_values, (res_n_lon,res_n_lat))
    
    return b_values, n_values

In [10]:
def processDayBlockMetadata(e):
    gix = lambda a : min(int((a - long_min) / gridsize), res_n_lon-1)
    giy = lambda a : min(int((a - lat_min ) / gridsize), res_n_lat-1)

    res_n_lon = int((long_max - long_min) / gridsize)
    res_n_lat = int((lat_max - lat_min) / gridsize)
    
    # New Selection because maxm and depth are supposed to be calculated on a day by day basis
    sel = np.logical_and(
            np.logical_and(
                catalog_dat > e - np.timedelta64(1, 'D'),
                catalog_dat < e - np.timedelta64(0, 'D')),
            np.logical_and(catalog_dep/1000 <= depth1, 
                           catalog_dep/1000 > depth0))
    
    s_mags = catalog_mag[sel]
    s_depth = catalog_dep[sel]
    s_long = catalog_lon[sel]
    s_lats = catalog_lat[sel]

    depth_values = np.ones((res_n_lon,res_n_lat), dtype='float32')*np.nan
    maxmag = np.ones((res_n_lon,res_n_lat), dtype='float16') * -1.0

    for i in range(len(s_long)):
        if maxmag[gix(s_long[i]), giy(s_lats[i])] < s_mags[i]:
            maxmag[gix(s_long[i]), giy(s_lats[i])] = s_mags[i] 
            depth_values[gix(s_long[i]), giy(s_lats[i])] = s_depth[i]

    print('Done With {}!'.format(e), end='\r')
    return maxmag, depth_values

# 1. Create Single Block
## 1.1 Set Parameters &  Functions

In [27]:
long_min = 120.0
long_max = 150.0
lat_min  = 20.0
lat_max  = 50.0

gridsize = 0.1
dr       = 0.6  # 0.15, 0.35, 0.6
lookback = 730   # 150, 365, 730

depth0 = -1
depth1 = 70

calcB = calcBplus # set which function to use
metadata = False   # create Metadata for file. Includes calculation and saving it to file


## 1.2 Create Sliced Data

In [ ]:
fname = blockdir + 'Tiles_Parameters_0.1_sliced.hdf5'
dates = np.arange('2000-01-01', '2023-01-01', dtype='datetime64[D]')

dr_gl       = dr
lookback_gl = lookback

tx1 = time.time()
with Pool(96) as p: 
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        res = p.map(processDayBlock, dates)
    f1 = h5py.File(fname, 'a')
    for i, r in enumerate(res):
        groupname = 'day_{:04d}_{}'.format(i, str(dates[i]))
        if groupname in f1:
            g1 = f1[groupname]
        else:
            g1 = f1.create_group('day_{:04d}_{}'.format(i, str(dates[i])))
        print('\n StepX')
        g1.create_dataset('b_value_{:03d}_{:4.2f}'.format(lookback, dr), r[0].shape, dtype='float16', data=r[0])
        g1.create_dataset(   'n_eq_{:03d}_{:4.2f}'.format(lookback, dr), r[1].shape, dtype='int16',   data=r[1])
        if metadata:
            g1.create_dataset('maxmag', r[0].shape, dtype='float16', data=r[2])
            g1.create_dataset('depth',  r[1].shape, dtype='float32', data=r[3])
    f1.close()
tx2 = time.time()

print('Calculation: {} s'.format(t2-t1))

with open(masterdir + 'log.preprocessing', 'a') as fo:
    fo.write('Finished the d = {:03d} r = {:4.2f} run. Runtime: {} s'.format(lookback, dr, tx2-tx1))

/tmp/ipykernel_2285469/2550802568.py:7: RuntimeWarning: invalid value encountered in less_equal
  np.logical_and(catalog_dep/1000 <= depth1,
/tmp/ipykernel_2285469/2550802568.py:7: RuntimeWarning: invalid value encountered in less_equal
  np.logical_and(catalog_dep/1000 <= depth1,
/tmp/ipykernel_2285469/2550802568.py:7: RuntimeWarning: invalid value encountered in less_equal
  np.logical_and(catalog_dep/1000 <= depth1,
/tmp/ipykernel_2285469/2550802568.py:7: RuntimeWarning: invalid value encountered in less_equal
  np.logical_and(catalog_dep/1000 <= depth1,
/tmp/ipykernel_2285469/2550802568.py:7: RuntimeWarning: invalid value encountered in less_equal
  np.logical_and(catalog_dep/1000 <= depth1,
/tmp/ipykernel_2285469/2550802568.py:7: RuntimeWarning: invalid value encountered in less_equal
  np.logical_and(catalog_dep/1000 <= depth1,
/tmp/ipykernel_2285469/2550802568.py:7: RuntimeWarning: invalid value encountered in less_equal
  np.logical_and(catalog_dep/1000 <= depth1,
/tmp/ipykerne

## 1.3 Merge slices into blocks

In [13]:
f2 = h5py.File(blockdir + 'Tiles_Parameters_0.1.hdf5', 'a') # append to add new data in case of parametersearches
f3 = h5py.File(blockdir + 'Tiles_Parameters_0.1_sliced.hdf5', 'r')

dates = np.arange('2000-01-01', '2023-01-01', dtype='datetime64[D]')

keys = f3.keys()
keys = sorted(np.array([str(key) for key in keys]))

fullBlock_b = np.zeros((len(keys), 300, 300), dtype='float16')
fullBlock_n = np.zeros((len(keys), 300, 300), dtype=int)
fullBlock_maxmag = np.zeros((len(keys), 300, 300), dtype='float16')
fullBlock_depth = np.zeros((len(keys), 300, 300), dtype='float32')

for i, key in enumerate(keys):
    fullBlock_b[i] = np.array(f3[key+'/b_value_{:03d}_{:4.2f}'.format(lookback, dr)])
    fullBlock_n[i] = np.array(f3[key+'/n_eq_{:03d}_{:4.2f}'.format(lookback, dr)])
    if metadata:
        fullBlock_maxmag[i] = np.array(f3[key+'/maxmag']).T
        fullBlock_depth[i]  = np.array(f3[key+'/depth']).T
f3.close()

f2.create_dataset('b_value_{:03d}_{:4.2f}'.format(lookback, dr), fullBlock_b.shape, dtype='float16', data=fullBlock_b)
f2.create_dataset(   'n_eq_{:03d}_{:4.2f}'.format(lookback, dr), fullBlock_n.shape, dtype='int16',   data=fullBlock_n)
f2.create_dataset('maxmag',                                      fullBlock_b.shape, dtype='float16', data=fullBlock_maxmag)
f2.create_dataset('depth',                                       fullBlock_n.shape, dtype='float32', data=fullBlock_depth)
f2.close()


## 1.4 Make NoEQ Array Conditions

In [3]:
days_before = 7
days_after  = 7
maglim      = 4.5
l1distance  = 8 # in steps of gridsize
mean_neq    = 10

In [ ]:
f2 = h5py.File(blockdir + 'Tiles_Parameters_0.1.hdf5', 'r')
n_eq_avg = np.array(f2['n_eq_{:03d}_{:4.2f}'.format(lookback, dr)])
maxm_loc = np.array(f2['maxmag'])
dept_avg = np.array(f2['depth'])
f2.close()

def makeNoEQArray():
    t1 = time.time()
    noeqarray = np.zeros(maxm_loc.shape-np.array([512, 32,32]), dtype=bool)
    for i in range(512, maxm_loc.shape[0]):
        for j in range(16, maxm_loc.shape[1]-16):
            for k in range(16, maxm_loc.shape[2]-16):
                if np.max(maxm_loc[i-7:i+7, j-8:j+8, k-8:k+8]) < 4.5:
                    if np.mean(n_eq_avg[i-7:i+7, j-8:j+8, k-8:k+8]) > 10:
                        noeqarray[i-512, j-16, k-16] = True
        t2 = time.time()
        t_rem = (t2 - t1) / (i - 512) * (maxm_loc.shape[0] - 512)
        print('Daterange: 512 - {} | @ {} | Runtime: {}s Remaining: {}s       '.format(maxm_loc.shape[0], i, int(t2-t1), int(t_rem)), end='\r')
    return noeqarray

t1 = time.time()
noeqarray = makeNoEQArray()
t2 = time.time()
print('\nCalculation: {} s'.format(t2-t1))

f2 = h5py.File(blockdir + 'Tiles_Parameters_0.1.hdf5', 'a')
f2.create_dataset('NoEQArray_{:03d}_{:4.2f}_{:3.1f}'.format(lookback, dr, maglim), noeqarray.shape, dtype='float16', data=noeqarray)
f2.close()

# 2. Create many blocks with varying parameters

In [8]:
long_min = 120.0
long_max = 150.0
lat_min  = 20.0
lat_max  = 50.0

gridsize = 0.1
dr       = [0.15, 0.25, 0.35, 0.60] 
lookback = [150, 365, 730] 

depth0 = -1
depth1 = 70

calcB = calcBplus # set which function to use
metadata = True   # create Metadata for file. Includes calculation and saving it to file


## 2.1 Calculate b, n_eq (and metadata)
### 2.1.1 Actual b value Calculation

In [6]:
def process_one_parameterconfig(lookback, dr, metadata):
    fname = blockdir + 'Tiles_Parameters_0.1_sliced.hdf5'
    dates = np.arange('2000-01-01', '2023-01-01', dtype='datetime64[D]')
    
    tx1 = time.time()
    with Pool(96) as p: 
        res = p.map(processDayBlock, dates)
        f1 = h5py.File(fname, 'a')
        print('processDayBlock finished!')
        
        for i, r in enumerate(res):
            groupname = 'day_{:04d}_{}'.format(i, str(dates[i]))
            if groupname in f1:
                g1 = f1[groupname]
            else:
                g1 = f1.create_group('day_{:04d}_{}'.format(i, str(dates[i])))

            g1.create_dataset('b_value_{:03d}_{:4.2f}'.format(lookback, dr), r[0].shape, dtype='float16', data=r[0])
            g1.create_dataset(   'n_eq_{:03d}_{:4.2f}'.format(lookback, dr), r[1].shape, dtype='int16',   data=r[1])
        f1.close()
    tx2 = time.time()
    
    print('\n Calculation: {} s'.format(tx2-tx1))
    
    with open(masterdir + 'log.preprocessing', 'a') as fo:
        fo.write('Finished the d = {:03d} r = {:4.2f} run. Runtime: {} s'.format(lookback, dr, tx2-tx1))

In [ ]:
for dr_i in dr:
    for lookback_i in lookback:
        lookback_gl = lookback_i
        dr_gl = dr_i
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            print('Starting Paramter Combination dr={} lookback={}'.format(dr_i, lookback_i))
            process_one_parameterconfig(lookback_i, dr_i, metadata)

        if metadata:
            metadata = False
        print('Finished Paramter Combination dr={} lookback={}'.format(dr_i, lookback_i))
        

Starting Paramter Combination dr=0.15 lookback=150
processDayBlock finished!

 Calculation: 5802.501230239868 s
Finished Paramter Combination dr=0.15 lookback=150
Starting Paramter Combination dr=0.15 lookback=365


In [ ]:
f1 = h5py.File(blockdir + 'Tiles_Parameters_0.1_sliced.hdf5', 'r')
for key in f1['day_0000_2000-01-01'].keys():
    print(key)

f1.close()

### 2.1.2 Metadata

In [12]:
def addMetadata():
    fname = blockdir + 'Tiles_Parameters_0.1_sliced.hdf5'
    dates = np.arange('2000-01-01', '2023-01-01', dtype='datetime64[D]')
    
    tx1 = time.time()
    with Pool(96) as p: 
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            res = p.map(processDayBlockMetadata, dates)
            
        f1 = h5py.File(fname, 'a')
        print('processDayBlock finished!')
        
        for i, r in enumerate(res):
            groupname = 'day_{:04d}_{}'.format(i, str(dates[i]))
            if groupname in f1:
                g1 = f1[groupname]
            else:
                g1 = f1.create_group('day_{:04d}_{}'.format(i, str(dates[i])))

            g1.create_dataset('maxmag', r[0].shape, dtype='float16', data=r[0])
            g1.create_dataset('depth',  r[1].shape, dtype='float32', data=r[1])
        f1.close()
    tx2 = time.time()
    
    print('\n Calculation: {} s'.format(tx2-tx1))
    
    with open(masterdir + 'log.preprocessing', 'a') as fo:
        fo.write('Finished Metadata run. Runtime: {} s'.format( tx2-tx1))

In [ ]:
addMetadata()

### 2.1.3 Merge sliced data into one block

The merging is fast (should not take more than 1 minute per parameterconfiguration). 

In [13]:
f2 = h5py.File(blockdir + 'Tiles_Parameters_0.1.hdf5', 'a') # append to add new data in case of parametersearches
f3 = h5py.File(blockdir + 'Tiles_Parameters_0.1_sliced.hdf5', 'r')

dates = np.arange('2000-01-01', '2023-01-01', dtype='datetime64[D]')

keys = f3.keys()
keys = sorted(np.array([str(key) for key in keys]))

sets_keys = [key for key in f3[keys[0]].keys() if 'b_value' in key]
apdxs = ['_'.join(key.split('_')[2:]) for key in sets_keys]

fullBlock_maxmag = np.zeros((len(keys), 300, 300), dtype='float16')
fullBlock_depth = np.zeros((len(keys), 300, 300), dtype='float32')

for i, key in enumerate(keys):
    fullBlock_maxmag[i] = np.array(f3[key+'/maxmag']).T
    fullBlock_depth[i]  = np.array(f3[key+'/depth']).T
    print('Metadata ' + key, end='\r')

f2.create_dataset('maxmag', fullBlock_maxmag.shape, dtype='float16', data=fullBlock_maxmag)
f2.create_dataset('depth',  fullBlock_depth.shape,  dtype='float32', data=fullBlock_depth)



for apdx in apdxs:
    fullBlock_b = np.zeros((len(keys), 300, 300), dtype='float16')
    fullBlock_n = np.zeros((len(keys), 300, 300), dtype=int)
    print('')
    for i, ikey in enumerate(keys):
        fullBlock_b[i] = np.array(f3[ikey+'/b_value_'+apdx])
        fullBlock_n[i] = np.array(f3[ikey+'/n_eq_'+apdx])
        print(apdx + ' ' + key, end='\r')
        
    f2.create_dataset('b_value_'+apdx, fullBlock_b.shape, dtype='float16', data=fullBlock_b)
    f2.create_dataset(   'n_eq_'+apdx, fullBlock_n.shape, dtype='int16',   data=fullBlock_n)

f3.close()
f2.close()


Metadata day_8400_2022-12-31
150_0.15 day_8400_2022-12-31
150_0.25 day_8400_2022-12-31
150_0.35 day_8400_2022-12-31
150_0.60 day_8400_2022-12-31
365_0.15 day_8400_2022-12-31
365_0.25 day_8400_2022-12-31
365_0.35 day_8400_2022-12-31
365_0.60 day_8400_2022-12-31
730_0.15 day_8400_2022-12-31
730_0.25 day_8400_2022-12-31
730_0.35 day_8400_2022-12-31


## 2.2 Calculate NoEqArray for many parameters
### 2.2.1 Binary classification

In [6]:
def parallelNoEqAr(args):
    lookback = args['lookback']
    dr = args['dr']
    maglim = args['maglim']

    print('Args {} started!\n'.format(str(args)), end='')

    
    f2 = h5py.File(blockdir + 'Tiles_Parameters_0.1.hdf5', 'r')
    n_eq_avg = np.array(f2['n_eq_{:03d}_{:4.2f}'.format(lookback, dr)])
    maxm_loc = np.array(f2['maxmag'])
    f2.close()
    
    t1 = time.time()
    noeqarray = np.zeros(maxm_loc.shape-np.array([512, 32,32]), dtype=bool)

    print(' --> Args {} has loaded! \n'.format(str(args)), end='')

    
    for i in range(512, maxm_loc.shape[0]):
        for j in range(16, maxm_loc.shape[1]-16):
            for k in range(16, maxm_loc.shape[2]-16):
                if np.max(maxm_loc[i-7:i+7, j-8:j+8, k-8:k+8]) <= maglim:
                    if np.mean(n_eq_avg[i-7:i+7, j-8:j+8, k-8:k+8]) > 10:
                        noeqarray[i-512, j-16, k-16] = True
        if i%10 == 0:
            t2 = time.time()
            t_rem = (t2 - t1) / (i - 511) * (maxm_loc.shape[0] - 512)
            report = 'Run: {:03d}_{:4.2f} | Daterange: 512 - {} | @ {} | Runtime: {}s Remaining: {}s \n'.format(lookback, dr, maxm_loc.shape[0], i, int(t2-t1), int(t_rem))
            with open('logs/log_{:03d}_{:4.2f}'.format(lookback, dr), 'a') as fo:
                fo.write(report)

    return noeqarray


In [7]:
arglist = [
    {'lookback':150, 'dr':0.15, 'maglim':4.0},
    {'lookback':150, 'dr':0.15, 'maglim':4.5},
    {'lookback':150, 'dr':0.15, 'maglim':4.9},
    {'lookback':150, 'dr':0.25, 'maglim':4.0},
    {'lookback':150, 'dr':0.25, 'maglim':4.5},
    {'lookback':150, 'dr':0.25, 'maglim':4.9},
    {'lookback':150, 'dr':0.35, 'maglim':4.0},
    {'lookback':150, 'dr':0.35, 'maglim':4.5},
    {'lookback':150, 'dr':0.35, 'maglim':4.9},
    {'lookback':150, 'dr':0.60, 'maglim':4.0},
    {'lookback':150, 'dr':0.60, 'maglim':4.5},
    {'lookback':150, 'dr':0.60, 'maglim':4.9},
    {'lookback':365, 'dr':0.15, 'maglim':4.0},
    {'lookback':365, 'dr':0.15, 'maglim':4.5},
    {'lookback':365, 'dr':0.15, 'maglim':4.9},
    {'lookback':365, 'dr':0.25, 'maglim':4.0},
    {'lookback':365, 'dr':0.25, 'maglim':4.5},
    {'lookback':365, 'dr':0.25, 'maglim':4.9},
    {'lookback':365, 'dr':0.35, 'maglim':4.0},
    {'lookback':365, 'dr':0.35, 'maglim':4.5},
    {'lookback':365, 'dr':0.35, 'maglim':4.9},
    {'lookback':365, 'dr':0.60, 'maglim':4.0},
    {'lookback':365, 'dr':0.60, 'maglim':4.5},
    {'lookback':365, 'dr':0.60, 'maglim':4.9},
    {'lookback':730, 'dr':0.15, 'maglim':4.0},
    {'lookback':730, 'dr':0.15, 'maglim':4.5},
    {'lookback':730, 'dr':0.15, 'maglim':4.9},
    {'lookback':730, 'dr':0.25, 'maglim':4.0},
    {'lookback':730, 'dr':0.25, 'maglim':4.5},
    {'lookback':730, 'dr':0.25, 'maglim':4.9},
    {'lookback':730, 'dr':0.35, 'maglim':4.0},
    {'lookback':730, 'dr':0.35, 'maglim':4.5},
    {'lookback':730, 'dr':0.35, 'maglim':4.9},
    {'lookback':730, 'dr':0.60, 'maglim':4.0},
    {'lookback':730, 'dr':0.60, 'maglim':4.5},
    {'lookback':730, 'dr':0.60, 'maglim':4.9},
]

In [ ]:
tx1 = time.time()
with Pool(36) as p:
    res = p.map(parallelNoEqAr, arglist)

print('Parallel Processing Done')
f2 = h5py.File(blockdir + 'Tiles_Parameters_0.1.hdf5', 'a')
for i, r in enumerate(res):
    lookback = arglist[i]['lookback']
    dr       = arglist[i]['dr']
    maglim   = arglist[i]['maglim']
    f2.create_dataset('NoEQArray_d{:03d}_r{:4.2f}_m{:3.1f}'.format(lookback, dr, maglim), r.shape, dtype='float16', data=r)
f2.close()

tx2 = time.time()

print('Writing Done. {}s'.format(int(tx2-tx1)))

Args {'lookback': 150, 'dr': 0.15, 'maglim': 4.0} started!
Args {'lookback': 150, 'dr': 0.25, 'maglim': 4.0} started!
Args {'lookback': 150, 'dr': 0.25, 'maglim': 4.9} started!
Args {'lookback': 150, 'dr': 0.15, 'maglim': 4.5} started!
Args {'lookback': 150, 'dr': 0.25, 'maglim': 4.5} started!
Args {'lookback': 150, 'dr': 0.15, 'maglim': 4.9} started!
Args {'lookback': 150, 'dr': 0.35, 'maglim': 4.0} started!
Args {'lookback': 150, 'dr': 0.35, 'maglim': 4.9} started!
Args {'lookback': 150, 'dr': 0.6, 'maglim': 4.9} started!
Args {'lookback': 150, 'dr': 0.35, 'maglim': 4.5} started!
Args {'lookback': 150, 'dr': 0.6, 'maglim': 4.0} started!
Args {'lookback': 365, 'dr': 0.15, 'maglim': 4.0} started!
Args {'lookback': 365, 'dr': 0.35, 'maglim': 4.0} started!
Args {'lookback': 365, 'dr': 0.15, 'maglim': 4.5} started!
Args {'lookback': 150, 'dr': 0.6, 'maglim': 4.5} started!
Args {'lookback': 365, 'dr': 0.25, 'maglim': 4.9} started!
Args {'lookback': 365, 'dr': 0.25, 'maglim': 4.0} started!


# 3. Merge Day Slices

In [14]:
f2 = h5py.File(blockdir + 'Tiles_Parameters_0.1.hdf5', 'a') # append to add new data in case of parametersearches
f3 = h5py.File(blockdir + 'Tiles_Parameters_0.1_sliced.hdf5', 'r')

dates = np.arange('2000-01-01', '2023-01-01', dtype='datetime64[D]')

keys = f3.keys()
keys = sorted(np.array([str(key) for key in keys]))

sets_keys = [key for key in f3[keys[0]].keys() if 'b_value' in key]
apdxs = ['_'.join(key.split('_')[2:]) for key in sets_keys]


fullBlock_maxmag = np.zeros((len(keys), 300, 300), dtype='float16')
fullBlock_depth = np.zeros((len(keys), 300, 300), dtype='float32')

for i, key in enumerate(keys):
    fullBlock_maxmag[i] = np.array(f3[key+'/maxmag']).T
    fullBlock_depth[i]  = np.array(f3[key+'/depth']).T
    print('Metadata ' + key, end='\r')

f2.create_dataset('maxmag', fullBlock_maxmag.shape, dtype='float16', data=fullBlock_maxmag)
f2.create_dataset('depth',  fullBlock_depth.shape,  dtype='float32', data=fullBlock_depth)



for apdx in apdxs:
    fullBlock_b = np.zeros((len(keys), 300, 300), dtype='float16')
    fullBlock_n = np.zeros((len(keys), 300, 300), dtype=int)
    print('')
    for i, ikey in enumerate(keys):
        fullBlock_b[i] = np.array(f3[ikey+'/b_value_'+apdx])
        fullBlock_n[i] = np.array(f3[ikey+'/n_eq_'+apdx])
        print(apdx + ' ' + key, end='\r')
        
    f2.create_dataset('b_value_'+apdx, fullBlock_b.shape, dtype='float16', data=fullBlock_b)
    f2.create_dataset(   'n_eq_'+apdx, fullBlock_n.shape, dtype='int16',   data=fullBlock_n)

f3.close()
f2.close()


Metadata day_8247_2022-07-31
150_0.15 day_8247_2022-07-31
150_0.25 day_8247_2022-07-31
150_0.35 day_8247_2022-07-31
150_0.60 day_8247_2022-07-31
365_0.15 day_8247_2022-07-31
365_0.25 day_8247_2022-07-31
365_0.35 day_8247_2022-07-31
365_0.60 day_8247_2022-07-31
730_0.15 day_8247_2022-07-31
